In [1]:
# Import libraries
import pandas as pd
# import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
# import requests
# import time
import scipy.stats as st
# from pathlib import Path

# CSV file-paths
fantasy_file = "./Resources/fantasy_merged_7_17.csv"
fantasy_adp_file = "./Resources/adp_merged_7_17.csv"

fantasy_stats = pd.read_csv(fantasy_file) # 3388 x 27
fantasy_adp = pd.read_csv(fantasy_adp_file) # 1206 x 6

In [2]:
# Merge both DFs together and fill all NaNs with the value 0
fantasy_merged = pd.merge(fantasy_stats, fantasy_adp, how="left", on=["PlayerID", "Year"])

In [3]:
# Data Cleaning
# --------------------------
# Fill all NaN values with 0
fantasy_merged_clean = fantasy_merged.fillna(0) # 3388 x 31
fantasy_stats_clean = fantasy_stats.fillna(0)

# capture '2TM' and '3TM' in separate DF and view (players that were traded to another team during year)
multi_team_df = fantasy_stats_clean.loc[(fantasy_stats_clean["Tm"]=="2TM") | (fantasy_stats_clean["Tm"]=="3TM")]
# filter out multi_team_df using .isin()
filtered_team_df = fantasy_stats_clean[~fantasy_stats_clean.Tm.isin(multi_team_df.Tm)]

# Replace 'OAK' and 'LVR' values in 'Tm' column with 'RAI' (Raiders location changed from OAK to LVR in 2020)
cleaned_team_df = filtered_team_df.copy()
cleaned_team_df.loc[cleaned_team_df["Tm"]=="OAK", "Tm"] = "RAI"
cleaned_team_df.loc[cleaned_team_df["Tm"]=="LVR", "Tm"] = "RAI"

In [4]:
#fantasy_merged_clean.sort_values(by=["Year", "adp"], ascending=[True, False])
#fantasy_merged_clean.to_csv("./Resources/merged_clean.csv")

# 2) Which teams had the players with the overall highest fantasy rank per year?

In [5]:
# Filter for 'Rk' = 1
cleaned_team_df.loc[cleaned_team_df["Rk"]==1, ["Rk", "PPR", "Player", "Tm", "Year"]].reset_index(drop=True)

,Rk,PPR,Player,Tm,Year
0,1,383.3,Todd Gurley,LAR,2017
1,1,417.1,Patrick Mahomes,KAN,2018
2,1,471.2,Christian McCaffrey,CAR,2019
3,1,396.1,Josh Allen,BUF,2020
4,1,439.5,Cooper Kupp,LAR,2021
5,1,417.4,Patrick Mahomes,KAN,2022


In [6]:
# 2.1) Check dropped row percentage of multi_team_df against fantasy_stats_clean
dropped_row_pct = (multi_team_df["Rk"].count()/fantasy_stats_clean["Rk"].count()) * 100
print(f"The percentage of dropped rows of 'fantasy_stats_clean' is {round(dropped_row_pct,2)}%.")

The percentage of dropped rows of 'fantasy_stats_clean' is 3.93%.


# Data Cleaning

In [7]:
# Fantasy rank is "Rk" (also based on "PPR" values for each year)
# Highest fantasy rank would be lowest average of 'Rk' column
# use fantasy_stats_clean
# Columns 'Rk', 'Tm', 'Year'

In [8]:
fantasy_merged_clean

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,Fmb,FL,PPR,PlayerID,PosRk,Year,name,position,team,adp
0,1,Todd Gurley,LAR,RB,23,15,15,0,0,0,...,5,2,383.3,GurlTo01,1.0,2017,Todd Gurley,RB,LAR,19.4
1,2,Russell Wilson,SEA,QB,29,16,16,339,553,3983,...,14,3,347.9,WilsRu00,1.0,2017,Russell Wilson,QB,SEA,64.3
2,3,Le'Veon Bell,PIT,RB,25,15,15,0,0,0,...,3,2,341.6,BellLe00,2.0,2017,Le'Veon Bell,RB,PIT,2.3
3,4,Alvin Kamara,NOR,RB,22,16,3,0,0,0,...,1,1,320.4,KamaAl00,3.0,2017,Alvin Kamara,RB,NO,140.0
4,5,Antonio Brown,PIT,WR,29,14,14,0,0,0,...,4,0,310.3,BrowAn04,1.0,2017,Antonio Brown,WR,PIT,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,573,Nate Sudfeld,DET,QB,29,2,0,0,0,0,...,0,0,-0.4,SudfNa00,80.0,2022,0,0,0,0.0
3384,574,KaVontae Turpin,DAL,WR,26,17,0,0,0,0,...,3,2,-0.4,TurpKa00,218.0,2022,0,0,0,0.0
3385,575,Chad Henne,KAN,QB,37,3,0,0,2,0,...,1,0,-0.5,HennCh01,81.0,2022,0,0,0,0.0
3386,576,C.J. Beathard,JAX,QB,29,4,0,7,11,35,...,0,0,-1.0,BeatC.00,82.0,2022,0,0,0,0.0


In [9]:
fantasy_merged_clean.groupby("Year").max()["adp"]

C:\Users\jason\AppData\Local\Temp\ipykernel_4200\806991889.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  fantasy_merged_clean.groupby("Year").max()["adp"]


Year
2017    162.9
2018    166.8
2019    166.9
2020    164.2
2021    172.7
2022    153.8
Name: adp, dtype: float64

In [10]:
#for year in years:
fantasy_merged_clean.loc[(fantasy_merged_clean["Year"]==2017) & (fantasy_merged_clean["adp"]==162.9), :]

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,Fmb,FL,PPR,PlayerID,PosRk,Year,name,position,team,adp
75,76,Mohamed Sanu,ATL,WR,28,15,15,1,1,51,...,0,0,174.3,SanuMo00,29.0,2017,Mohamed Sanu,WR,ATL,162.9


In [11]:
fantasy_stats_clean["Tm"].unique()

array(['LAR', 'SEA', 'PIT', 'NOR', 'HOU', 'CAR', 'NWE', 'KAN', 'LAC',
       'PHI', 'WAS', 'DET', 'BUF', 'ARI', 'DAL', 'MIA', 'ATL', 'JAX',
       'MIN', 'SFO', 'CIN', 'GNB', 'CLE', 'TEN', 'OAK', 'DEN', 'IND',
       'NYJ', 'TAM', 'CHI', 'NYG', 'BAL', '2TM', '3TM', 'LVR'],
      dtype=object)

In [12]:
fantasy_stats_clean["Tm"].nunique()
# 35 unique values; there are 32 NFL teams
# Raiders were OAK until 2019; changed to LVR 2020; change both to RAI?
# figure out what to do with '2TM' and '3TM' (127 and 6)

35

In [13]:
fantasy_stats_clean["Tm"].value_counts()

2TM    127
NOR    116
WAS    112
NYJ    111
DET    108
JAX    107
GNB    107
NYG    107
TEN    106
SEA    105
TAM    104
DEN    104
BAL    104
HOU    103
ATL    102
IND    102
MIA    102
ARI    102
CLE    101
LAC    101
CHI    101
SFO    100
PHI     99
PIT     99
NWE     98
CAR     98
BUF     97
KAN     97
CIN     96
DAL     91
LAR     89
MIN     89
LVR     49
OAK     48
3TM      6
Name: Tm, dtype: int64

In [14]:
fantasy_stats_clean["Tm"].count()
# (127 + 6)/3388 * 100 = 3.9%; not too significant of a drop

3388

In [15]:
# capture '2TM' and '3TM' in separate DF and view
multi_team_df = fantasy_stats_clean.loc[(fantasy_stats_clean["Tm"]=="2TM") | (fantasy_stats_clean["Tm"]=="3TM")]
multi_team_df[multi_team_df["Tm"]=='2TM'] # 133 x 27

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,Rec,RecYds,YR,RecTD,Fmb,FL,PPR,PlayerID,PosRk,Year
119,120,Kelvin Benjamin,2TM,WR,26,14,14,0,0,0,...,48,692,14.42,3,0,0,135.2,BenjKe00,47.0,2017
120,121,Jay Ajayi,2TM,RB,24,14,8,0,0,0,...,24,158,6.58,1,3,2,135.1,AjayJa00,36.0,2017
158,159,Deonte Thompson,2TM,WR,28,16,10,0,0,0,...,38,555,14.61,2,1,0,105.0,ThomDe04,64.0,2017
189,190,Andre Ellington,2TM,RB,28,12,2,0,0,0,...,39,369,9.46,0,0,0,87.4,ElliAn00,54.0,2017
202,203,Adrian Peterson,2TM,RB,32,10,7,0,0,0,...,11,70,6.36,0,3,2,78.9,PeteAd01,61.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3307,497,Jonathan Ward,2TM,RB,25,8,0,0,0,0,...,2,7,3.50,0,0,0,5.2,WardJo00,142.0,2022
3313,503,Jaelon Darden,2TM,WR,23,14,0,0,0,0,...,2,26,13.00,0,0,0,4.8,DardJa00,185.5,2022
3323,513,Andy Isabella,2TM,WR,26,5,0,0,0,0,...,2,21,10.50,0,0,0,4.2,IsabAn00,191.0,2022
3352,542,Tyron Johnson,2TM,WR,26,4,1,0,0,0,...,1,8,8.00,0,0,0,1.8,JohnTy03,206.0,2022


In [16]:
# filter out using .isin()
filtered_team_df = fantasy_stats_clean[~fantasy_stats_clean.Tm.isin(multi_team_df.Tm)]
filtered_team_df # 3255 x 27

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,Rec,RecYds,YR,RecTD,Fmb,FL,PPR,PlayerID,PosRk,Year
0,1,Todd Gurley,LAR,RB,23,15,15,0,0,0,...,64,788,12.31,6,5,2,383.3,GurlTo01,1.0,2017
1,2,Russell Wilson,SEA,QB,29,16,16,339,553,3983,...,0,0,0.00,0,14,3,347.9,WilsRu00,1.0,2017
2,3,Le'Veon Bell,PIT,RB,25,15,15,0,0,0,...,85,655,7.71,2,3,2,341.6,BellLe00,2.0,2017
3,4,Alvin Kamara,NOR,RB,22,16,3,0,0,0,...,81,826,10.20,5,1,1,320.4,KamaAl00,3.0,2017
4,5,Antonio Brown,PIT,WR,29,14,14,0,0,0,...,101,1533,15.18,9,4,0,310.3,BrowAn04,1.0,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,573,Nate Sudfeld,DET,QB,29,2,0,0,0,0,...,0,0,0.00,0,0,0,-0.4,SudfNa00,80.0,2022
3384,574,KaVontae Turpin,DAL,WR,26,17,0,0,0,0,...,1,9,9.00,0,3,2,-0.4,TurpKa00,218.0,2022
3385,575,Chad Henne,KAN,QB,37,3,0,0,2,0,...,0,0,0.00,0,1,0,-0.5,HennCh01,81.0,2022
3386,576,C.J. Beathard,JAX,QB,29,4,0,7,11,35,...,0,0,0.00,0,0,0,-1.0,BeatC.00,82.0,2022


In [17]:
filtered_team_df["Tm"].value_counts()

NOR    116
WAS    112
NYJ    111
DET    108
NYG    107
GNB    107
JAX    107
TEN    106
SEA    105
TAM    104
DEN    104
BAL    104
HOU    103
MIA    102
IND    102
ATL    102
ARI    102
CHI    101
CLE    101
LAC    101
SFO    100
PHI     99
PIT     99
NWE     98
CAR     98
BUF     97
KAN     97
CIN     96
DAL     91
LAR     89
MIN     89
LVR     49
OAK     48
Name: Tm, dtype: int64

In [18]:
# Replace 'OAK' and 'LVR' in a DF with 'RAI' (Raiders location changed from OAK to LVR in 2020)
cleaned_team_df = filtered_team_df.copy()
cleaned_team_df.loc[cleaned_team_df["Tm"]=="OAK", "Tm"] = "RAI"
cleaned_team_df.loc[cleaned_team_df["Tm"]=="LVR", "Tm"] = "RAI"
cleaned_team_df["Tm"].value_counts()

NOR    116
WAS    112
NYJ    111
DET    108
NYG    107
GNB    107
JAX    107
TEN    106
SEA    105
BAL    104
TAM    104
DEN    104
HOU    103
MIA    102
IND    102
ATL    102
ARI    102
CLE    101
LAC    101
CHI    101
SFO    100
PHI     99
PIT     99
NWE     98
CAR     98
BUF     97
RAI     97
KAN     97
CIN     96
DAL     91
MIN     89
LAR     89
Name: Tm, dtype: int64

In [19]:
grouped_year_team_df = cleaned_team_df.groupby(["Year","Tm"])
grouped_year_team_df.describe()["Rk"]

count        mean         std   min     25%    50%     75%    max
Year Tm                                                                    
2017 ARI   19.0  289.842105  125.582316  20.0  212.50  267.0  381.00  529.0
     ATL   14.0  244.357143  179.709427  26.0   78.50  221.0  397.25  522.0
     BAL   17.0  256.058824  135.179728  67.0  118.00  260.0  368.00  463.0
     BUF   14.0  298.785714  156.867499  18.0  222.25  311.0  429.00  501.0
     CAR   14.0  266.642857  158.879058   7.0  160.50  292.0  365.00  520.0
...         ...         ...         ...   ...     ...    ...     ...    ...
2022 SEA   19.0  301.736842  191.332707  13.0  163.50  284.0  487.00  570.0
     SFO   17.0  292.588235  164.626114  42.0  161.00  289.0  422.00  565.0
     TAM   18.0  281.833333  180.567162  25.0  128.75  305.0  436.00  561.0
     TEN   18.0  315.111111  162.877395  14.0  181.50  359.5  456.75  556.0
     WAS   19.0  288.789474  158.127086  41.0  150.50  330.0  407.50  563.0

[192 rows x 8 columns]

In [20]:
#2017 Average Rank Values ('Rk') per team
cleaned_team_df[cleaned_team_df["Year"]==2017].groupby("Tm").mean(numeric_only=True)["Rk"]#.sort_values()

Tm
ARI    289.842105
ATL    244.357143
BAL    256.058824
BUF    298.785714
CAR    266.642857
CHI    288.812500
CIN    276.733333
CLE    300.722222
DAL    261.800000
DEN    262.588235
DET    268.352941
GNB    297.947368
HOU    303.800000
IND    288.800000
JAX    287.684211
KAN    269.000000
LAC    261.666667
LAR    264.500000
MIA    262.933333
MIN    237.133333
NOR    276.625000
NWE    203.642857
NYG    245.375000
NYJ    270.500000
PHI    246.647059
PIT    281.058824
RAI    213.642857
SEA    262.764706
SFO    229.857143
TAM    227.250000
TEN    280.466667
WAS    256.764706
Name: Rk, dtype: float64

In [21]:
years = cleaned_team_df["Year"].unique()
#years[0].astype('str')
#str(years[0])

In [22]:
test_df = pd.DataFrame({}) # create empty DataFrame
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame


In [23]:
for year in years:
    test_df[str(year) + " Avg Player Rank"] = cleaned_team_df[cleaned_team_df["Year"]==year].groupby("Tm").mean(numeric_only=True)["Rk"]

In [24]:
test_df

,2017 Avg Player Rank,2018 Avg Player Rank,2019 Avg Player Rank,2020 Avg Player Rank,2021 Avg Player Rank,2022 Avg Player Rank
Tm,,,,,,
ARI,289.842105,305.125000,287.133333,264.466667,294.411765,309.000000
ATL,244.357143,272.117647,251.941176,294.944444,321.611111,301.333333
BAL,256.058824,251.941176,255.875000,293.176471,303.055556,288.894737
BUF,298.785714,333.111111,293.529412,265.823529,253.400000,292.000000
CAR,266.642857,277.294118,303.812500,329.526316,312.470588,291.000000
CHI,288.812500,283.705882,279.750000,299.555556,275.125000,328.166667
CIN,276.733333,263.125000,242.357143,286.777778,296.647059,280.625000
CLE,300.722222,255.062500,286.411765,286.058824,278.277778,277.866667
DAL,261.800000,277.142857,212.076923,251.600000,247.941176,282.764706


In [25]:
test_df.min()

2017 Avg Player Rank    203.642857
2018 Avg Player Rank    235.500000
2019 Avg Player Rank    212.076923
2020 Avg Player Rank    191.833333
2021 Avg Player Rank    247.941176
2022 Avg Player Rank    220.384615
dtype: float64

In [26]:
test_df.index

Index(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN',
       'DET', 'GNB', 'HOU', 'IND', 'JAX', 'KAN', 'LAC', 'LAR', 'MIA', 'MIN',
       'NOR', 'NWE', 'NYG', 'NYJ', 'PHI', 'PIT', 'RAI', 'SEA', 'SFO', 'TAM',
       'TEN', 'WAS'],
      dtype='object', name='Tm')

In [27]:
for year in years:
    print(test_df[str(year) + " Avg Player Rank"].sort_values().head())

Tm
NWE    203.642857
RAI    213.642857
TAM    227.250000
SFO    229.857143
MIN    237.133333
Name: 2017 Avg Player Rank, dtype: float64
Tm
TAM    235.500000
LAR    236.000000
SFO    238.058824
LAC    243.142857
NWE    243.714286
Name: 2018 Avg Player Rank, dtype: float64
Tm
DAL    212.076923
KAN    232.437500
SFO    240.133333
CIN    242.357143
LAR    248.428571
Name: 2019 Avg Player Rank, dtype: float64
Tm
LAR    191.833333
PIT    231.214286
MIN    235.928571
TAM    242.117647
DAL    251.600000
Name: 2020 Avg Player Rank, dtype: float64
Tm
DAL    247.941176
NWE    250.375000
TAM    253.222222
BUF    253.400000
SEA    254.250000
Name: 2021 Avg Player Rank, dtype: float64
Tm
PHI    220.384615
KAN    240.500000
NYJ    254.352941
MIN    260.333333
JAX    261.000000
Name: 2022 Avg Player Rank, dtype: float64
